## Analysis of European options using Merton's jump-diffusion model

In this mini-project, we explore Merton's jump-diffusion model (MJD) as a way of describing the evolution of stock paths. First, we set up the theory, explaining the underlying stochastic differential equation and solving it. Then, we derive a closed-form expression for the value of a European call option. Finally, we investigate the effect of delta hedging.

### The stochastic differential equation

We begin from the well-known geometric Brownian motion (GBM) model with constant volatility:
$$
\begin{equation} \tag{GBM-SDE}
    dS_t = (\mu+r)S_t dt + \sigma S_t dW_t ,
\end{equation}
$$
where $S_t$ is the asset price at time $t$, and $W_t = \sqrt{t} \mathcal{N}(0,1)$ is the Weiner process (mean $0$ and variance $t$). The drift is given by $\mu$, while $\sigma$ is the volatility and $r$ is the risk-neutral interest rate. The solution to this stochastic differential equation (SDE) is 
$$
\begin{equation} \tag{GBM}
    S_t = S_0 e^{(\mu+r -\frac{1}{2}\sigma^2)t + \sigma W_t} .
\end{equation}
$$

One of the shortcomings of this simplistic model is its failure to describe sudden market rises and crashes. It also does not display the "fat tails" that are often seen with real data. To fix these problems, Merton suggested the following "jump-diffusion" correction:
$$
\begin{equation*}
    dS_t = (\mu+r)S_t dt + \sigma S_t dW_t + (Y - 1)S_t dN_t .
\end{equation*}
$$
Here, $Y S_t$ is the new asset price after a sidden "kick", so $(Y - 1)$ is the percentage change in the asset price due to the kick. The random variable $N_t$ counts the number of such kicks in the time interval $[0,t]$ (more precisely, $dN_t = 1$ with probability $\lambda dt$ and $dN_t = 0$ with probability $1 - \lambda dt$, where $\lambda$ is a rate parameter.). The jump size $Y$ is log-normally distributed, i.e., $\ln(Y) \sim \mathcal{N}(\alpha,\delta^2)$; the jump count $N_t$ is a Poisson process, i.e., $N_t \sim$ Poisson $(\lambda dt)$. 

Both of these random variables $(Y, N_t)$ are independent of each other, which means $E[(Y-1)dN_t] = E[Y-1] E[dN_t] = k \times \lambda dt$, where $k = e^{\alpha + \frac{\delta^2}{2}} - 1$. We subtract out this jump-induced drift. Also, if there are multiple jumps in the time interval $dt$, then the jump term will be $\Pi_{j=1}^{dN_t} Y_{j} - 1$. After incorporating these changes, we have
$$
\begin{equation} \tag{MJD-SDE} 
    d(\ln S_t) = (\mu+r - \lambda k) dt + \sigma dW_t + (\Pi_{j=1}^{dN_t} Y_{j} - 1) .
\end{equation}
$$

Using Ito's lemma (for Poisson processes), the solution may be written as
$$
\begin{equation*}
    S_t = S_0 e^{\left(\mu + r - \lambda k - \frac{1}{2}\sigma^2 \right)t + \sigma W_t + \sum_{j=1}^n \ln Y_{j} } ,
\end{equation*}
$$
where the number of jumps $n \sim$ Poisson $(\lambda dt)$. Owing to the independence of the $\ln Y_j$'s and $W_t$, we arrive at the asset pricing under the jump-diffusion model:
$$
\begin{equation} \tag{MJD}
    S_t = S_0 e^{\left(\mu + r - \lambda k - \frac{1}{2}\sigma^2 \right)t + \mathcal{N}(n\alpha,\sigma^2 t + n\delta^2) }.
\end{equation}
$$

In [ ]:
import numpy as np
import math
import pandas as pd
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.optimize import minimize, newton
from scipy.stats import shapiro
import scipy.stats as stats
from scipy.optimize import brentq
from scipy.integrate import quad
from dateutil import parser
from dateutil.tz import tzutc

sns.set_style('darkgrid')

In [ ]:
# function to generate GBM paths
def GBM_paths(S0, mu, r, sigma, T, n_steps, n_sims):
    '''
    Generates Geometric Brownian Motion paths.
    
    Parameters:
    S0 (float) : Initial stock price
    mu (float) : Drift term
    r (float) : Risk-free interest rate
    sigma (float) : Volatility of the stock
    T (float) : Total time horizon
    n_steps (int) : Number of time steps
    n_sims (int) : Number of simulation paths
    
    Returns:
    np.ndarray : Simulated GBM paths of shape (n_steps + 1, n_sims)
    '''

    noise = np.random.normal(loc = 0, scale = 1, size =  (n_steps, n_sims))
    dt = T/n_steps
    
    log_returns = (mu + r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*noise
    exponent = np.cumsum(log_returns, axis = 0)
    
    paths = S0 * np.exp(exponent)
    paths_from_start = np.insert(paths, 0, S0, axis = 0)

    return paths_from_start


# function to generate MJD paths
def MJD_paths(S0, mu, r, sigma, lam, alpha, delta, k, T, n_steps, n_sims):
    '''
    Generates Merton Jump Diffusion paths.
    
    Parameters:
    S0 (float) : Initial stock price
    mu (float) : Drift term
    r (float) : Risk-free interest rate
    sigma (float) : Volatility of the stock
    lam (float) : Jump intensity (average number of jumps per unit time)
    alpha (float) : Mean of jump size (in log terms)
    delta (float) : Standard deviation of jump size (in log terms)
    k (float) : Expected relative jump size (E[e^Y - 1])
    T (float) : Total time horizon
    n_steps (int) : Number of time steps
    n_sims (int) : Number of simulation paths
    '''
    
    dt = T/n_steps
    n = np.random.poisson(lam * dt, size = (n_steps, n_sims))

    noise = np.random.normal(loc = 0, scale = 1, size = (n_steps, n_sims))
    
    log_returns = (mu + r - lam*k - 0.5*(sigma**2))*dt + n*alpha + np.sqrt((sigma**2)*dt + n*(delta**2))*noise
    exponent = np.cumsum(log_returns, axis = 0)
    
    paths = S0 * np.exp(exponent)
    paths_from_start = np.insert(paths, 0, S0, axis = 0)

    return paths_from_start

We now simulate some sample paths for the MJD, to illustrate the jump processes at play.

In [ ]:
# parameters for MJD paths
S0 = 100
mu = 0.05
r = 0.03
sigma = 0.2
lam = 0.5
alpha = -0.1
delta = 0.5
k = np.exp(alpha + 0.5*delta**2) - 1

T = 1
n_steps = 252
n_sims = 5

paths = MJD_paths(S0, mu, r, sigma, lam, alpha, delta, k, T, n_steps, n_sims)

# plot the paths
plt.figure(figsize=(10,6))
for i in range(n_sims):
    plt.plot(paths[:, i], lw=1, alpha=0.5)
plt.plot(np.mean(paths, axis=1), color='black', lw=2, label='Mean Path')
plt.title(f'Merton Jump Diffusion with {n_sims} paths')
plt.xlabel('Time steps')
plt.ylabel('Asset price')
plt.legend()
plt.show()

Next, we compare the mean paths generated by the MJD and the GBM models. The former clearly displays sudden jumps, which lead to much larger fluctuations of the asset price.

In [ ]:
# comparing MJD and GBM mean paths

# parameters
S0 = 100
mu = 0
r = 0.03
sigma = 0.2
lam = 1
alpha = -0.2
delta = 0.5
k = np.exp(alpha + 0.5*delta**2) - 1

T = 1
n_steps = 252
n_sims = 1000

path_MJD = MJD_paths(S0, mu, r, sigma, lam, alpha, delta, k, T, n_steps, n_sims)
path_GBM = GBM_paths(S0, mu, r, sigma, T, n_steps, n_sims)

# plot the paths
plt.figure(figsize=(10,6))
plt.plot(np.mean(path_MJD, axis=1), color='black', lw=1, label='Mean MJD Path')
plt.plot(np.mean(path_GBM, axis=1), color='red', lw=1, label='Mean GBM Path')
plt.title(f'MJD vs GBM, averaged across {n_sims} paths')
plt.xlabel('Time steps')
plt.ylabel('Asset price')
plt.legend()
plt.show()

### Pricing a call option
We now move onto the task of pricing a call option under the MJD model. To this end, we will recast the asset pricing solution in a slightly different form, so that we may use the Black-Scholes result directly.
$$
\begin{equation*} 
    S_t = S_0 e^{\left(\mu + r - \lambda k - \frac{1}{2}\sigma^2 \right)t + \mathcal{N}(n\alpha,\sigma^2 t + n\delta^2) } = S_0^{(n)} e^{\left(\mu' + r - \frac{1}{2}\sigma_n^2 \right)t + \sigma_n W_t } ,
\end{equation*}
$$
where $S_0^{(n)} = S_0 e^{n\left(\alpha + \frac{\delta^2}{2} \right)}$, $\mu' = \mu - \lambda k$, and $\sigma_n^2 = \sigma^2 + \frac{n\delta^2}{t}$. Since the RHS resembles a GBM (with modified values of the initial stock price, drift and volatility), we obtain the fair market call option value (conditioned on the number of jumps $n$) to be
$$
\begin{equation*} \tag{Call conditioned on $n$}
    C_0(N_T = n) = S_0^{(n)} e^{\mu' t} \Phi(d_1) - K e^{-rt} \Phi(d_2) ,
\end{equation*}
$$
where $K$ is the strike price, $\Phi$ is the cumulative distribution function of the standard normal distribution, and
$$
\begin{equation*}
    d_1 = \frac{\ln\left(\frac{S_0^{(n)}}{K}\right) + \left(\mu' + r + \frac{\sigma_n^2}{2}\right)t}{\sigma_n\sqrt{t}}, \text{ and } d_2 = d_1 - \sigma_n\sqrt{t}.
\end{equation*}
$$
Finally, we use the Poisson distribution for $n$, to deduce the fair-market call option price to be
$$
\begin{equation*} \tag{Call price}
    C_0 = \sum_{n=0}^{\infty} C_0(N_T = n)\cdot P(n) = e^{-\lambda T}\sum_{n=0}^{\infty} \frac{(\lambda T)^n}{n!} C_0(N_T = n) .
\end{equation*}
$$

Now, we define the necessary functions to evaluate this option price.

In [ ]:
# function to calculate fair-market call price using B-S model
def BS_call(S0, mu, r, sigma, t, K):
    '''
    Black-Scholes Call Option formula
    
    Inputs:
    S0 (float): Stock price at time 0
    K (float): Strike Price
    sigma: Yearly volatility
    t: Time to expiration (years)
    r: Risk-free Interest rate
    mu: Expected return of the underlying asset
    
    Return:
    Black-Scholes value of call option (float)
    '''
    
    d1 = (np.log(S0/K) + (mu + r + (0.5)*sigma**2)*t)/(sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)
    
    BS_call_value = S0*(np.exp(mu*t))*norm.cdf(d1) - K*np.exp(-r*t)*norm.cdf(d2)
    
    return BS_call_value

In [24]:
# function to calculate fair-market call price using MJD model
def MJD_call(S0, mu, r, sigma, lam, alpha, delta, k, t, K):
    '''
    MJD Call Option formula
    
    Inputs:
    S0 (float): Stock price at time 0
    K (float): Strike Price
    sigma: Yearly volatility
    t: Time to expiration (years)
    r: Risk-free Interest rate
    mu: Expected return of the underlying asset
    lam: Jump intensity
    alpha: Mean of jump size
    delta: Std dev of jump size
    k: Expected percentage jump size
    
    Return:
    MJD value of call option (float)
    '''
    
    MJD_call_value = 0

    mu1 = mu - lam*k

    for n in range(0, 51): # sum from n=0 to 50, as higher n terms contribute negligibly
        sigma_n = np.sqrt(sigma**2 + (n * delta**2)/t)
        S0_n = S0 * np.exp(n * (alpha + 0.5 * delta**2))  # Adjusted stock price for n jumps

        MJD_call_value += BS_call(S0_n, mu1, r, sigma_n, t, K)*((lam*t)**n)/math.factorial(n)
    
    return np.exp(-lam*t)*MJD_call_value

Next, we simulate some paths using the MJD model, and evaluate the call option in two ways: using the fundamental definition
$$
\begin{equation*}
    C_0 = e^{-rt} E[\max(S_T - K, 0)] ,
\end{equation*}
$$
and using the option pricing formula derived above.

In [ ]:
# comparing call option pricing using MJD formula vs simulation

# parameters for MJD paths
S0 = 100
mu = 0.05
r = 0.03
sigma = 0.2
lam = 0.5
alpha = -0.1
delta = 0.5
k = np.exp(alpha + 0.5*delta**2) - 1

T = 1
n_steps = 252
n_sims = 10000

paths = MJD_paths(S0, mu, r, sigma, lam, alpha, delta, k, T, n_steps, n_sims)

# simulate call option payoffs at maturity
K = 110
payoffs = np.maximum(paths[-1, :] - K, 0)
simulated_call_price = np.exp(-r*T) * np.mean(payoffs)

MJD_formula_price = MJD_call(S0, mu, r, sigma, lam, alpha, delta, k, T, K)

print(f"Call option price according to simulation = ${simulated_call_price:.2f}")
print(f"Call option price according to MJD formula = ${MJD_formula_price:.2f}")


### Delta and hedging

We calculate (and simulate) the delta $\Delta_{C_0}$ for the MJD evolution. By differentiating (Call price) with respect to $S_0$, we obtain
$$
\begin{equation*}
    \Delta_{C_0} = e^{(\mu' -\lambda) T} \sum_{n=0}^{\infty} \frac{(\lambda T)^n}{n!} \Phi(d_1) e^{n\left( \alpha + \frac{\delta^2}{2} \right)} = e^{(\mu' -\lambda) T} \sum_{n=0}^{\infty} \frac{(\lambda T (1+k))^n}{n!} \Phi(d_1),
\end{equation*}
$$
where (we recall that) $d_1 = \frac{\ln\left(\frac{S_0^{(n)}}{K}\right) + \left(\mu' + r + \frac{\sigma_n^2}{2}\right)t}{\sigma_n\sqrt{t}}$, $S_0^{(n)} = S_0 e^{n\left(\alpha + \frac{\delta^2}{2} \right)}$, and $\Phi$ is the cumulative distribution function of the standard normal random variable.

In [ ]:
# function to calculate Black-Scholes Delta of Call Option
def BS_call_delta(S0, mu, r, sigma, t, K):
    '''Black-Scholes Delta of Call Option
    
    Parameters:
    S0 (float): Stock price at time 0
    K (float): Strike Price
    sigma: Yearly volatility
    t: Time to expiration (years)
    r: Risk-free Interest rate
    mu: Expected return of the underlying asset

    Return:
    Black-Scholes rate of change of call option with respect to S_0
    
    '''
    
    
    d1 = (np.log(S0/K) + (mu + r + (0.5)*sigma**2)*t)/(sigma*np.sqrt(t))
    
    
    return np.exp(mu*t)*norm.cdf(d1)


# function to calculate MJD Delta of Call Option
def MJD_call_delta(S0, mu, r, sigma, lam, alpha, delta, k, t, K):
    '''
    MJD Delta of Call Option
    
    Inputs:
    S0 (float): Stock price at time 0
    K (float): Strike Price
    sigma: Yearly volatility
    t: Time to expiration (years)
    r: Risk-free Interest rate
    mu: Expected return of the underlying asset
    lam: Jump intensity
    alpha: Mean of jump size
    delta: Std dev of jump size
    k: Expected percentage jump size
    
    
    Return:
    MJD rate of change of call option with respect to S_0 (float)
    '''
    mu1 = mu - lam*k

    MJD_call_delta_value = 0

    for n in range(0, 101): # sum from n=0 to 100, as higher n terms contribute negligibly
        sigma_n = np.sqrt(sigma**2 + (n * delta**2)/t) # adjusted volatility
        S0_n = S0 * np.exp(n * (alpha + 0.5 * delta**2))  # Adjusted stock price for n jumps

        MJD_call_delta_value += BS_call_delta(S0_n, mu1, r, sigma_n, t, K)*((lam*t*(1+k))**n)/math.factorial(n)
    
    return np.exp((-lam)*t)*MJD_call_delta_value

In [ ]:
# simulating delta of call option under MJD model

# parameters for MJD paths
S0 = 100
mu = 0.05
r = 0.03
sigma = 0.2
lam = 0.5
alpha = -0.1
delta = 0.5
k = np.exp(alpha + 0.5*delta**2) - 1
K = 110

T = 1
n_steps = 252
n_sims = 1

paths = MJD_paths(S0, mu, r, sigma, lam, alpha, delta, k, T, n_steps, n_sims)
path = paths[:, 0]

t = np.linspace(0,T, n_steps + 1)

Deltas = MJD_call_delta(path[:n_steps], mu, r, sigma, lam, alpha, delta, k, (T-t)[:n_steps], K)

timeaxis_paths = np.arange(len(path))
timeaxis_deltas = np.arange(len(Deltas))


# plotting asset price path, strike price, and delta
fig, ax1 = plt.subplots(figsize=(10, 8))

ax1.set_xlabel('Time steps')
ax1.set_ylabel('Asset price', color = 'black')
plot1 = ax1.plot(timeaxis_paths, path, label = 'Asset price', color = 'black')
plotK = ax1.axhline(K, label = 'Strike price', color = 'red')
ax1.tick_params(axis='y', labelcolor = 'black')
ax1.grid(True, alpha=0.3)

ax2 = ax1.twinx()

ax2.set_ylabel('Delta', color= 'orange') # We already handled the x-label with ax1
plot2 = ax2.plot(timeaxis_deltas, Deltas, label = 'Delta', color = 'orange')
ax2.tick_params(axis='y', labelcolor = 'orange')

plots = plot1 + plot2 + [plotK]
labels = [p.get_label() for p in plots]

fig.legend(plots, labels, loc='lower center', ncol=3)
plt.title(f'Delta of call option under MJD model (K = {K})')
fig.tight_layout()  # Adjusts plot to prevent labels from overlapping
plt.subplots_adjust(bottom=0.1) # Make space for the legend at the bottom
plt.show()

The "asymptotic" behavior of $\Delta_{C_0}$ (to either $0$ or $K$), depending on whether the asset price is above or below the strike price as we approach the time of expiration of the call option, is similar to what is observed in the GBM model. However, upon closer inspection (see below), we notice that the $\Delta_{C_0}$ usually saturates to its final value earlier in the GBM model as compared to the MJD model. This is in line with the intuition that the MJD model allows for kicks that could suddenly jolt the system, even as the expiration draws closer.

In [ ]:
# comparing the Delta behavior between MJD and GBM models

# parameters
S0 = 100
mu = 0.05
r = 0.03
sigma = 0.2
lam = 0.5
alpha = -0.1
delta = 0.5
k = np.exp(alpha + 0.5*delta**2) - 1
K = 110

T = 1
n_steps = 252
n_sims = 1

path_MJD = MJD_paths(S0, mu, r, sigma, lam, alpha, delta, k, T, n_steps, n_sims)[:,0]
path_GBM = GBM_paths(S0, mu, r, sigma, T, n_steps, n_sims)[:,0]

t = np.linspace(0,T, n_steps + 1)

Deltas_MJD = MJD_call_delta(path_MJD[:n_steps], mu, r, sigma, lam, alpha, delta, k, (T-t)[:n_steps], K)
Deltas_GBM = BS_call_delta(path_GBM[:n_steps], mu, r, sigma, (T-t)[:n_steps], K)

# adjusting the x-axis for number of time steps instead of time in years
timeaxis_paths = np.arange(len(path_MJD))
timeaxis_deltas = np.arange(len(Deltas_MJD))

# plotting asset price path, strike price, and delta
fig, ax1 = plt.subplots(figsize=(10, 8))

ax1.set_xlabel('Time steps')
ax1.set_ylabel('Asset price', color = 'black')
plot1a = ax1.plot(timeaxis_paths, path_MJD, label = 'Asset price (MJD)', color = 'black')
plot1b = ax1.plot(timeaxis_paths, path_GBM, label = 'Asset price (GBM)', color = 'black', ls = '--')
plotK = ax1.axhline(K, label = 'Strike price', color = 'red')
ax1.tick_params(axis='y', labelcolor = 'black')
ax1.grid(True, alpha=0.3)

ax2 = ax1.twinx()

ax2.set_ylabel('Delta', color= 'orange') # We already handled the x-label with ax1
plot2a = ax2.plot(timeaxis_deltas, Deltas_MJD, label = 'Delta (MJD)', color = 'orange')
plot2b = ax2.plot(timeaxis_deltas, Deltas_GBM, label = 'Delta (GBM)', color = 'orange', ls = '--')
ax2.tick_params(axis='y', labelcolor = 'orange')

plots = plot1a + plot1b + plot2a + plot2b + [plotK]
labels = [p.get_label() for p in plots]

fig.legend(plots, labels, loc='lower center', ncol=3)
plt.title(f'Delta of call option: MJD vs GBM (K = {K})')
fig.tight_layout()  # Adjusts plot to prevent labels from overlapping
plt.subplots_adjust(bottom=0.15) # Make space for the legend at the bottom
plt.show()

Having tested our routines for calculating $\Delta_{C_0}$, it is now time to simulate some Delta hedging. We will present the market maker's perspective, i.e., someone who is selling call options and hedging by buying/shorting shares of the same asset.

In [ ]:
# Delta hedging using MJD (from a market maker's perspective)

# parameters
S0 = 100
mu = 0.05
r = 0.03
sigma = 0.2
lam = 0.5
alpha = -0.1
delta = 0.5
k = np.exp(alpha + 0.5*delta**2) - 1
K = 90
premium = 15

T = 1
n_steps = 252*20    #20 hedges per day!
n_sims = 1

# expected profit from MJD call option formula
MJD_call_price = MJD_call(S0, mu, r, sigma, lam, alpha, delta, k, T, K)
expected_profit = premium - MJD_call_price
print(f"Expected profit (no hedging) = ${expected_profit:.2f}")

# simulated profit from MJD path (no hedging)
paths_MJD = MJD_paths(S0, mu, r, sigma, lam, alpha, delta, k, T, n_steps, n_sims)[:,0]
payoffs = np.maximum(paths_MJD[-1] - K, 0)
simulated_call_payoff = np.exp(-r*T) * np.mean(payoffs)
simulated_profit = premium - simulated_call_payoff
print(f"Simulated profit (no hedging) = ${simulated_profit:.2f}")

# simulated profit from MJD path with delta hedging
t = np.linspace(0,T, n_steps + 1)
Deltas_MJD = MJD_call_delta(paths_MJD[:n_steps], mu, r, sigma, lam, alpha, delta, k, (T-t)[:n_steps], K)

dt = T/n_steps
stock_profits_discounted = (paths_MJD[1:n_steps + 1] - \
                            paths_MJD[0:n_steps]*np.exp(r*dt))*np.exp(-r*t[1:n_steps+1])*Deltas_MJD

total_stock_profits = np.sum(stock_profits_discounted)
profit_of_call_with_hedging = simulated_call_payoff - total_stock_profits
hedged_portfolio_profit = premium - profit_of_call_with_hedging
print(f'Simulated profit with {n_steps} hedges: ${hedged_portfolio_profit:.2f}')

# plot the stock path
plt.figure(figsize = (8,6))
plt.plot(np.arange(len(paths_MJD)), paths_MJD, label = 'Asset price (MJD)', color = 'black')
plt.axhline(K, label = 'Strike price', color = 'red')
plt.title('Asset price path')
plt.xlabel('Time steps')
plt.ylabel('Asset price')
plt.legend()
plt.show()

We can also repeat this process for many realizations of the simulated path and look at the distribution of the portfolio profits. As the following simulation confirms (see the last plot), hedging decreases one's exposure by ameliorating the harshest losses. However, this happens at the expense of a reduced maximum profit. In other words, there's no free lunch! 

It is noteworthy that this improvement due to hedging is not as dramatic in the case of the MJD as it was in the GBM model. So, for MJD, any hedging is better than no hedging, but more hedging is not necessarily much better. It appears to be a system with diminishing returns (unlike GBM, which keeps "improving" with more hedging). This is because MJD allows for random kicks which could knock the asset price so far off, that even hedging does not help much. This is the reason for the fatter tails in the MJD model, which is also something that is observed in the real markets.

Another important point is that the average profit of the hedged portfolio (with non-zero drift) is very close to the expected profit of the _hedging-free and drift-free_ case! This is a demonstration of how increased hedging can help to offset the effect of drift of an asset.

In [ ]:
# Delta hedging using MJD (from a market maker's perspective) across many realizations

# parameters
S0 = 100
mu = 0.05
r = 0.03
sigma = 0.2
lam = 0.5
alpha = -0.2
delta = 0.5
k = np.exp(alpha + 0.5*delta**2) - 1
K = 100
premium = 20

T = 1
hedges = [0, 1, 7, 28, 84, 168, 252, 2520]
n_sims = 5000

# expected profit from MJD call option formula (WITH ZERO DRIFT!)
MJD_call_price = MJD_call(S0, 0, r, sigma, lam, alpha, delta, k, T, K)
expected_portfolio_profit = premium - MJD_call_price

min_profit = []
max_profit = []

for n_steps in hedges:

    if n_steps == 0:
        paths_MJD = MJD_paths(S0, mu, r, sigma, lam, alpha, delta, k, T, 1, n_sims)
        payoffs = np.maximum(paths_MJD[-1,:] - K, 0)
        simulated_call_payoff = np.exp(-r*T) * np.mean(payoffs)
        simulated_portfolio_profit = premium - payoffs
        average_simulated_portfolio_profit = np.mean(simulated_portfolio_profit)
        
        # calculating min and max profits of unhedged portfolio
        min_simulated_portfolio_profit = np.min(simulated_portfolio_profit)
        max_simulated_portfolio_profit = np.max(simulated_portfolio_profit)

        # plotting histogram of unhedged portfolio profits
        plt.figure(figsize = (8,6))
        plt.hist(simulated_portfolio_profit, bins = 'auto')
        plt.axvline(expected_portfolio_profit, color = 'red', label = f'Expected portfolio profit: ${expected_portfolio_profit:.2f}')
        plt.axvline(average_simulated_portfolio_profit, ls = '--', color = 'black', label = f'Average simulated profit: ${average_simulated_portfolio_profit:.2f}')
        plt.plot([], [], ' ', label=f'Minimum profit = ${min_simulated_portfolio_profit:.2f}')
        plt.plot([], [], ' ', label=f'Maximum profit = ${max_simulated_portfolio_profit:.2f}')
        plt.title(f'Profit distribution of call option portfolio with {n_steps} hedges')
        plt.xlabel('Profit ($)')
        plt.legend()
        plt.show()

    else:
        # simulated profit from MJD path (no hedging)
        paths_MJD = MJD_paths(S0, mu, r, sigma, lam, alpha, delta, k, T, n_steps, n_sims)
        payoffs = np.maximum(paths_MJD[-1,:] - K, 0)
        simulated_call_payoff = np.exp(-r*T) * np.mean(payoffs)
        simulated_portfolio_profit = premium - simulated_call_payoff

        # simulated profit from MJD path with delta hedging
        t = np.linspace(0,T, n_steps + 1)
        tte = (T - t)[0:n_steps]
        tte = tte.reshape(-1,1)
        times = t.reshape(-1,1)
        Deltas_MJD = MJD_call_delta(paths_MJD[:n_steps,:], mu, r, sigma, lam, alpha, delta, k, tte, K)

        dt = T/n_steps
        stock_profits_discounted = (paths_MJD[1:n_steps + 1,:] - \
                                    paths_MJD[:n_steps,:]*np.exp(r*dt))*np.exp(-r*times[1:n_steps+1])*Deltas_MJD

        total_stock_profits = np.sum(stock_profits_discounted, axis = 0)
        profit_of_call_with_hedging = payoffs - total_stock_profits
        hedged_portfolio_profit = premium - profit_of_call_with_hedging
        average_hedged_portfolio_profit = np.mean(hedged_portfolio_profit)
        
        # calculating min and max profits of hedged portfolio
        min_hedged_portfolio_profit = np.min(hedged_portfolio_profit)
        max_hedged_portfolio_profit = np.max(hedged_portfolio_profit)

        # appending min and max profits to lists
        min_profit.append(min_hedged_portfolio_profit)
        max_profit.append(max_hedged_portfolio_profit)


        # plotting histogram of hedged portfolio profits
        plt.figure(figsize = (8,6))
        plt.hist(hedged_portfolio_profit, bins = 'auto')
        plt.axvline(expected_portfolio_profit, color = 'red', label = f'Expected portfolio profit: ${expected_portfolio_profit:.2f}')
        plt.axvline(average_hedged_portfolio_profit, ls = '--', color = 'black', label = f'Average simulated profit: ${average_hedged_portfolio_profit:.2f}')
        plt.plot([], [], ' ', label=f'Minimum profit = ${min_hedged_portfolio_profit:.2f}')
        plt.plot([], [], ' ', label=f'Maximum profit = ${max_hedged_portfolio_profit:.2f}')
        plt.title(f'Profit distribution of call option portfolio with {n_steps} hedges')
        plt.xlabel('Profit ($)')
        plt.legend()
        plt.show()

# appending the unhedged min and max profits
min_profit = np.insert(min_profit, 0, min_simulated_portfolio_profit)
max_profit = np.insert(max_profit, 0, max_simulated_portfolio_profit) 

# plotting the minimum and maximum profits against number of hedges
fig, ax1 = plt.subplots(figsize=(10, 8))

ax1.set_xlabel('Number of hedges')
ax1.set_ylabel('Minimum portfolio profit', color = 'red')
plot1 = ax1.plot(hedges, min_profit, label = 'Minimum profit', color = 'red')
ax1.tick_params(axis='y', labelcolor = 'black')
ax1.grid(True, alpha=0.3)

ax2 = ax1.twinx()

ax2.set_ylabel('Maximum portfolio profit', color= 'blue') # We already handled the x-label with ax1
plot2 = ax2.plot(hedges, max_profit, label = 'Maximum profit', color = 'blue')
ax2.tick_params(axis='y', labelcolor = 'black')

plots = plot1 + plot2
labels = [p.get_label() for p in plots]

fig.legend(plots, labels, loc='lower center', ncol=2)
plt.title(f'Minimum and Maximum Portfolio Profits vs Number of Hedges')
fig.tight_layout()  # Adjusts plot to prevent labels from overlapping
plt.subplots_adjust(bottom=0.15) # Make space for the legend at the bottom
plt.show()